In [ ]:
import pandas as pd
import numpy as np
import wfdb
import ast
import warnings; warnings.filterwarnings('ignore')

def load_raw_data(df, sampling_rate, path):
    if sampling_rate == 100:
        data = [wfdb.rdsamp(path+f) for f in df.filename_lr]
    else:
        data = [wfdb.rdsamp(path+f) for f in df.filename_hr]
    data = np.array([signal for signal, meta in data])
    return data

# set filepath and sampling rate
path = 'C:/Users/sande/OneDrive/Documents/SKOLE/master/PTB-XL/ptb-xl-a-large-publicly-available-electrocardiography-dataset-1.0.3/'
sampling_rate = 100





In [ ]:
# read label file and transfer it
Y = pd.read_csv(path+'ptbxl_database.csv', index_col='ecg_id')
Y.scp_codes = Y.scp_codes.apply(lambda x: ast.literal_eval(x))

# Load raw signal data
X = load_raw_data(Y, sampling_rate, path)

# Load scp_statements.csv for diagnostic aggregation
scp = pd.read_csv(path+'scp_statements.csv', index_col=0)
scp = scp[scp.diagnostic == 1]

In [ ]:
def diagnostic_class(y_dic):
    tmp = []
    for key in y_dic.keys():
        if key in scp.index:
            tmp.append(scp.loc[key].diagnostic_class)
    return list(set(tmp))
    
# Apply diagnostic superclass
Y['target_class'] = Y.scp_codes.apply(diagnostic_class)
Y['target_class_len'] = Y['target_class'].apply(len)
Y.loc[Y.target_class_len > 1, 'target_class']

In [ ]:
all_class = pd.Series(np.concatenate(Y['target_class'].values))
class_cols = all_class.unique()
update_cols = np.concatenate([class_cols]) # add meta data columns
meta_cols = ['age', 'sex', 'height', 'weight', 'nurse', 'site', 'device',] # could add more columns as features

class ClassUpdate():
    def __init__(self, cols):
        self.cols = cols

    def __call__(self, row):
        for sc in row['target_class']:
            row[sc] = 1
        
            
        return row

def get_data_by_folds(folds, x, y, update_cols, feature_cols):
    assert len(folds)  > 0, '# of provided folds should longer than 1'
    filt = np.isin(y.strat_fold.values, folds)
    x_selected = x[filt]
    y_selected = y[filt]
    
    for sc in update_cols:
        y_selected[sc] = 0
        
    cls_updt = ClassUpdate(update_cols)
    
    y_selected = y_selected.apply(cls_updt, axis=1)
    
    return x_selected, y_selected[list(feature_cols)+list(update_cols)+['strat_fold']]

In [ ]:
x_all, y_all = get_data_by_folds(np.arange(1, 11), X, Y, update_cols, meta_cols)

x_train, y_train = get_data_by_folds(np.arange(1, 9), X, Y, update_cols, meta_cols)

x_valid, y_valid = get_data_by_folds([9], X, Y, update_cols, meta_cols)

x_test, y_test = get_data_by_folds([10], X, Y, update_cols, meta_cols)

In [ ]:
id_cols = ['ecg_id']
channel_cols = ['channel-{}'.format(i) for i in range(12)]
y_train.to_csv('C:/Users/sande/OneDrive/Documents/SKOLE/master/pre/train2_meta.csv', index=True)
y_train_signal = pd.DataFrame(columns=id_cols+channel_cols, index=np.arange(y_train.shape[0]*1000), dtype=np.float32)

ecg_ids = []
signals = []
for i, ecg_id in enumerate(y_train.index.values):
    y_train_signal.loc[i*1000:(i+1)*1000-1, 'ecg_id'] = [ecg_id]*1000
    y_train_signal.loc[i*1000:(i+1)*1000-1, channel_cols] = x_train[i,:,:]

y_train_signal['ecg_id'] = y_train_signal['ecg_id'].astype(np.int)
y_train_signal.to_csv('C:/Users/sande/OneDrive/Documents/SKOLE/master/pre/train2_signal.csv', index=False)

y_valid.to_csv('C:/Users/sande/OneDrive/Documents/SKOLE/master/pre/valid2_meta.csv', index=True)
y_valid_signal = pd.DataFrame(columns=id_cols+channel_cols, index=np.arange(y_valid.shape[0]*1000), dtype=np.float32)

ecg_ids = []
signals = []
for i, ecg_id in enumerate(y_valid.index.values):
    y_valid_signal.loc[i*1000:(i+1)*1000-1, 'ecg_id'] = [ecg_id]*1000
    y_valid_signal.loc[i*1000:(i+1)*1000-1, channel_cols] = x_valid[i,:,:]

y_valid_signal['ecg_id'] = y_valid_signal['ecg_id'].astype(np.int)
y_valid_signal.to_csv('C:/Users/sande/OneDrive/Documents/SKOLE/master/pre/valid2_signal.csv', index=False)


y_test.to_csv('C:/Users/sande/OneDrive/Documents/SKOLE/master/pre/test2_meta.csv', index=True)
y_test_signal = pd.DataFrame(columns=id_cols+channel_cols, index=np.arange(y_test.shape[0]*1000), dtype=np.float32)

ecg_ids = []
signals = []
for i, ecg_id in enumerate(y_test.index.values):
    y_test_signal.loc[i*1000:(i+1)*1000-1, 'ecg_id'] = [ecg_id]*1000
    y_test_signal.loc[i*1000:(i+1)*1000-1, channel_cols] = x_test[i,:,:]

y_test_signal['ecg_id'] = y_test_signal['ecg_id'].astype(np.int)
y_test_signal.to_csv('C:/Users/sande/OneDrive/Documents/SKOLE/master/pre/test2_signal.csv', index=False)

